**TFIDF VECTORIZAR,CLASSIFIER: MLKNN**

In [ ]:
#needed installs
!pip install scikit-learn==0.24.1
!pip install scikit-multilearn

#needed imports
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from skmultilearn.adapt import MLkNN
from sklearn.metrics import hamming_loss, accuracy_score

#mount drive
from google.colab import drive
drive.mount('/content/drive')
print('mount success')

#read from file
aspects_df = pd.read_csv('/content/drive/Shareddrives/EMOThreat/train_set_taskA.csv')
X = aspects_df["Sentences"]
y = np.asarray(aspects_df[aspects_df.columns[:7]])
  
# initializing TfidfVectorizer 
vetorizar = TfidfVectorizer(max_features=7000, max_df=0.85)
# fitting the tf-idf on the given data
vetorizar.fit(X)
  
# splitting the data to training and testing data set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.10, random_state=42)
  
# transforming the data
X_train_tfidf = vetorizar.transform(X_train)
X_test_tfidf = vetorizar.transform(X_test)

In [ ]:

# import warnings filter
from warnings import simplefilter
# ignore all future warnings
simplefilter(action='ignore', category=FutureWarning)

#using Multi-label kNN classifier  
import time 
from sklearn.model_selection import GridSearchCV
start=time.time()

mlknn_classifier = MLkNN(k=20)
mlknn_classifier.fit(X_train_tfidf, y_train)
predicted = mlknn_classifier.predict(X_test_tfidf)

print("Hamming Lossing:",hamming_loss(y_test, predicted))
print("Accuracy Score:",accuracy_score(y_test, predicted))
print('Training Time Taken: ',round(time.time()-start,0),'seconds')

In [ ]:
from sklearn.metrics import multilabel_confusion_matrix,classification_report
from sklearn.metrics import precision_score, recall_score, f1_score

print(classification_report(y_test, predicted))
print(multilabel_confusion_matrix(y_test, predicted))

from sklearn.metrics import accuracy_score
print("ACC: ",accuracy_score(predicted,y_test))

In [ ]:
import pandas as pd
aspects_df = pd.read_csv('/content/drive/Shareddrives/EMOThreat/test_set_taskA.csv')
X = aspects_df["Sentences"]
xx=X
vetorizar = TfidfVectorizer(max_features=7000, max_df=0.85)
# fitting the tf-idf on the given data
vetorizar.fit(X)
X = vetorizar.transform(X)
mlknn=mlknn_classifier.predict(X)
csv = pd.DataFrame(mlknn.toarray(),columns=["anger","disgust","fear","sadness","surprise","happiness","neutral"])
csv.insert(7,"Sentences",aspects_df["Sentences"],True)
csv.to_csv("pred_tfidf_mlknn.csv", index=False)